In [ ]:
import torch

In [ ]:
my_list = [[1,2,3,4],[5,6,7,8]]
tensors = torch.tensor(my_list)
tensors.shape, tensors.dtype,

(torch.Size([2, 4]), torch.int64)

In [ ]:
# designing first neurel network
import torch.nn as nn

# input 3 features (input neurons = input features, output neurons = classes)
input_tensors = torch.tensor([[0.01,0.05,-0.06]])

linear_layer = nn.Linear(
    in_features=3,
    out_features=2,
    bias=True
)

output = linear_layer(input_tensors)
print(output,"the more weighted features:", linear_layer.weight, "the more bias layer:", linear_layer.bias)


tensor([[0.4090, 0.1432]], grad_fn=<AddmmBackward0>) the more weighted features: Parameter containing:
tensor([[ 0.1539,  0.3358,  0.1683],
        [-0.5334, -0.1299,  0.1365]], requires_grad=True) the more bias layer: Parameter containing:
tensor([0.4008, 0.1632], requires_grad=True)


In [ ]:
# element wise multiplication
a= torch.tensor([[1,2,3]])
b= torch.tensor([[4,5,6]])
c = a*b

# matrix multiplication                           #  For matrix multiplication to work, the number of columns in the first tensor must equal the
                                                    # number of rows in the second tensor. In this case, both tensors have a shape of 2x3.
d= torch.tensor([[1,2,3],[4,5,6]])
e= torch.tensor([[4,5,6],[7,8,9]])
f = d @ e.T # Transpose tensor e

print(c, "the f value is here:",f)

tensor([[ 4, 10, 18]]) the f value is here: tensor([[ 32,  50],
        [ 77, 122]])


In [ ]:
# with more linear layer in sequence
import torch.nn as nn

n_features = 4
n_classes = 1

model = nn.Sequential(
    nn.Linear(in_features=n_features, out_features=10),    # more hidden layers more parameters and more capacity of the model
    nn.Linear(in_features=10, out_features=5,bias = True),
    nn.Linear(in_features=5, out_features=n_classes) ,
    nn.Sigmoid()
)


In [ ]:
# Correctly access parameters and their numel() method
parameters_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of parameters in the model: {parameters_count}")

## or
total = 0
for parameters in model.parameters():
    total += parameters.numel()
print("number of elements or parameters: ",total)

Number of parameters in the model: 111
number of elements or parameters:  111


# Activation function
   add the non linearity to model.(1. sigmoid: for binary classification.,,
   2. softmax: for multiclassclassification)

In [ ]:
import torch
import torch.nn as nn

# create input tensors
input_tensors = torch.tensor([[0.01,0.05,-0.06]])

# apply the softmax to the last dimension of input tensors
output = nn.Softmax(dim = -1)
output = output(input_tensors)

print(output)


tensor([[0.3363, 0.3501, 0.3136]])


# Loss functions
to update the weights and biases


In [ ]:
# actual value + model prediction(inputs) --> loss function => float which is output(y^ prediction  - y ground truth)
# loss = F(y,y^)
# one hot encoding concept (convert integer y to a tensors of zeros and one)
import torch.nn.functional as F
loss = F.one_hot(torch.tensor(0),num_classes=4)
print(loss)
# if y =1
loss = F.one_hot(torch.tensor(1),num_classes=4)
print(loss)
# if y=2
loss = F.one_hot(torch.tensor(2),num_classes=4)
print(loss)

# crossentropy loss function
from torch.nn import CrossEntropyLoss
criterion = CrossEntropyLoss()
scores = torch.tensor([-5.2,4.6,0.8])
one_hot_target = torch.tensor([1,0,0])
output = criterion(scores.double(), one_hot_target.double())
output



tensor([1, 0, 0, 0])
tensor([0, 1, 0, 0])
tensor([0, 0, 1, 0])


tensor(9.8222, dtype=torch.float64)

In [ ]:
from torch import optim
import torch.nn as nn
from torch.utils.data import dataloader
sgd  = optim.SGD(model.parameters(), lr = 0.0001, momentum = 0.95, weight_decay = 0.0001 )
adam  = optim.AdamW(model.parameters(), lr = 0.0001, weight_decay = 0.0001) # weight_decay = 0.0001 to prevent from overfitting
relu= nn.ReLU()
leaky_relu = nn.LeakyReLU(negative_slope= 0.05)
# training the loop
num_epochs = 5
for epoch in range(num_epochs):
  for data in dataloader:
    inputs_features, labels = data
    # set the gradient to zero
    sgd.zero_grad()
    # run a forward pass
    outputs = model(inputs_features)
    criterion = nn.CrossEntropyLoss()
    loss = criterion(outputs, labels)
    # backward propagation
    loss.backward()
    # update the weights and biases
    sgd.step()

In [ ]:
training_loss = 0
for inputs, labels in trainloader:
  # run the forward pass
  output = model(inputs)
  # compute the loss
  loss = criterion(outputs,labels)
  # backpropagation
  loss.backward()
  sgd.zero_grad() # initilizing zero gradient first
  # update the weights and biases
  sgd.step()
  # calculate and sum the total loss
  training_loss += loss.item()
  epoch_loss = training_loss/len(trainloader)
  print("epoch loss:",epoch_loss)


In [ ]:
# validation loss
validation_loss = 0.0
model.eval()
with torch.no_grad():
  for inputs, labels in validationloader:
    # run the forward pass
    output = model(inputs)
    # compute the loss
    loss = criterion(outputs,labels)
    # calculate and sum the total loss
    validation_loss += loss.item()
    epoch_loss = validation_loss/len(validationloader)

In [ ]:
# calculating accuracy
import torchmetrics

# create the accuracy metrics
metric = torchmetrics.Accuracy(task = "multiclass", num_classes = 3)
for features, labels in dataloader:
  # forward pass
  output = model(features)
  # compute batch accuracy
  metric.update(output, labels.argmax(dim= -1))
# compute accuracy of the whole epoch
epoch_accuracy = metric.compute()
# reset the metric for next epoch
metric.reset()

In [ ]:
# random search learning rate to update the hyperparameters tunning
factor = np.random.uniform(2,6)
lr = 10 ** (-factor)

# Pytorch datasets

In [16]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import torch

class WaterDataset(Dataset):
  def __init__(self, features, labels):
    super().__init__()
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.float32)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx]

# Load the data from the CSV file
df_full = pd.read_csv(r"/content/water_potability.csv").dropna() # Added dropna() to handle missing values

# Split the data into features and labels
features_full = df_full.drop('Potability', axis=1).values
labels_full = df_full['Potability'].values

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features_full, labels_full, test_size=0.2, random_state=42)

# Create TensorDatasets from the split data
train_dataset = TensorDataset(torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32))


# dataloader
data_loader_train = DataLoader(train_dataset, batch_size = 32, shuffle = True) # Increased batch size for better training
data_loader_test = DataLoader(test_dataset, batch_size = 32, shuffle = False) # No need to shuffle test data

# features, labels = next(iter(data_loader_train)) # next(iter(data_loader)) this is only for one batch
# print("the features of data:", features, "the labels of data", labels)

In [ ]:
# sequential model definition
import torch.nn as nn
model = nn.Sequential(
    nn.Linear(in_features=9, out_features=16),
    nn.ReLU(),
    nn.Linear(16,8),
    nn.PReLU(),
    nn.Linear(8,1),
    nn.Sigmoid(),
)

# class based model define

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.feature1 = nn.Linear(in_features=9, out_features=16)
    self.feature2 = nn.Linear(in_features=16, out_features=8)
    self.feature3 = nn.Linear(in_features=8, out_features=1)

  def forward(self,x):
    x= nn.functional.relu(self.feature1(x))
    x= nn.functional.relu(self.feature2(x))
    x= nn.functional.sigmoid(self.feature3(x))
    return x

net = Net()

# training loop
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.AdamW(net.parameters(), lr = 0.001) # net bcz we define custom model using class

for epoch in range(1000):
  for data in data_loader:
    inputs, labels = data
    # set the gradient to zero
    optimizer.zero_grad()
    # run a forward pass
    outputs = net(inputs)
    loss = criterion(outputs, labels.unsqueeze(1).float())
    # backward propagation
    loss.backward()
    # update the weights and biases
    optimizer.step()


# Model evaluation
from torchmetrics import Accuracy

accuracy = Accuracy(task = "binary")
net.eval()
with torch.no_grad():
  for inputs, labels in data_loader_test:
    # run the forward pass
    outputs = net(inputs)
    # predict probabilty
    preb = (outputs > 0.5).float()
    # compute batch accuracy
    accuracy.update(preb, labels.unsqueeze(1).float())
# compute accuracy of the whole epoch
epoch_accuracy = accuracy.compute()
# reset the metric for next epoch
accuracy.reset()



# **What is Weight Initialization?**

In a neural network, each layer has weights and biases.

Before training starts, these weights must be initialized (given starting values).

If we choose bad initial values, training can be slow, unstable, or even fail.
#🔹 2. Why is it Important?

If all weights = 0 → all neurons learn the same thing → no learning (symmetry problem).

If weights are too large → gradients explode.

If weights are too small → gradients vanish.

Good initialization → faster convergence, stable training.
#🔹 3. Common Initialization Methods in PyTorch
✅ Default (PyTorch automatically)

PyTorch already initializes weights when you create layers (nn.Linear, nn.Conv2d).

Usually uses Kaiming/He Uniform or Xavier Uniform, depending on the layer.

In [15]:
import torch
import torch.nn as nn
import torch.nn.init as init

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(10, 20)
        self.fc2 = nn.Linear(20, 1)

        # Apply custom initialization
        init.xavier_uniform_(self.fc1.weight)   # Xavier
        init.zeros_(self.fc1.bias)              # Bias = 0
        init.kaiming_normal_(self.fc2.weight)   # Kaiming
        init.constant_(self.fc2.bias, 0.1)      # Bias = 0.1

model = MyModel()



## **Loading images to pytorch**

In [ ]:
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128,128))
    ])
# displaying images
train_dataloader = DataLoader(train_data, batch_size = 32, shuffle = True)

image , label = next(iter(train_dataloader))
print(image.shape, label.shape)
# making shape in 128,128,3
image = image.squeeze.permute(1,2,0)
print("shape of image :", image.shape) # = (128,128,3)

# showing image
plt.imshow(image)
plt.show()

# data augmentation (generating more data)
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.RandomRotation(degrees = 15),
    transforms.ToTensor(),
    transforms.Resize((128,128)),
])

dataset_train = ImageFolder(
    root = "/content/train",
    transform = train_transform,
)


In [5]:
# building a cnn model
import torch.nn as nn

class Net(nn.Module):
  def __init__(self, num_classes):
    super().__init__()
    self.feature_extracter = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
        nn.ELU(),
        nn.BatchNorm2d(16),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
        nn.ELU(),
        nn.BatchNorm2d(32), # normalize the layer
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Dropout(p=0.25),
        nn.Flatten(),
        )
    self.classfier = nn.Linear(in_features=32*64*64, out_features=num_classes)
    # or we can made a function on in_features=32*64*64
    self.classfier = nn.Linear(in_features = self._get_flattened_size(), out_features=num_classes)
  def _get_flattened_size(self):
    x = torch.zeros(1, 3, 64, 64)  # Example input: 3 channels, 64x64 image
    x = self.conv1(x)
    x = self.pool(x)
    return x.numel()  # total elements in tensor

# a forward pass
  def forward(self, x):
    x = self.feature_extracter(x)
    x = self.classfier(x)
    return x


In [ ]:
import huggingface_hub as hfh

# Example: search for datasets containing "sentiment"
datasets_with_sentiment = [d.id for d in hfh.list_datasets() if "sentiment" in d.id]
print(datasets_with_sentiment)

HTTP Error 429 thrown while requesting GET https://huggingface.co/api/datasets?sort=trendingScore&cursor=eyIkb3IiOlt7InRyZW5kaW5nU2NvcmUiOjAsIl9pZCI6eyIkZ3QiOiI2ODBkOTkwY2MzOGViZWFlNzg4ZmQ2YTkifX0seyJ0cmVuZGluZ1Njb3JlIjp7IiRsdCI6MH19LHsidHJlbmRpbmdTY29yZSI6bnVsbH1dfQ%3D%3D
Retrying in 1s [Retry 1/20].
HTTP Error 429 thrown while requesting GET https://huggingface.co/api/datasets?sort=trendingScore&cursor=eyIkb3IiOlt7InRyZW5kaW5nU2NvcmUiOjAsIl9pZCI6eyIkZ3QiOiI2ODBkOTkwY2MzOGViZWFlNzg4ZmQ2YTkifX0seyJ0cmVuZGluZ1Njb3JlIjp7IiRsdCI6MH19LHsidHJlbmRpbmdTY29yZSI6bnVsbH1dfQ%3D%3D
Retrying in 2s [Retry 2/20].
HTTP Error 429 thrown while requesting GET https://huggingface.co/api/datasets?sort=trendingScore&cursor=eyIkb3IiOlt7InRyZW5kaW5nU2NvcmUiOjAsIl9pZCI6eyIkZ3QiOiI2ODBkOTkwY2MzOGViZWFlNzg4ZmQ2YTkifX0seyJ0cmVuZGluZ1Njb3JlIjp7IiRsdCI6MH19LHsidHJlbmRpbmdTY29yZSI6bnVsbH1dfQ%3D%3D
Retrying in 4s [Retry 3/20].
HTTP Error 429 thrown while requesting GET https://huggingface.co/api/datasets?sort=trendin

['tyqiangz/multilingual-sentiments', 'Youseff1987/multilingual-sentiment-dataset', 'omilab/hebrew_sentiment', 'google-research-datasets/poem_sentiment', 'stanfordnlp/sentiment140', 'community-datasets/tamilmixsentiment', 'mkeskin/turkish_movie_sentiment', 'community-datasets/urdu_sentiment_corpus', 'pythainlp/wisesight_sentiment', 'Recognai/sentiment-banking', 'Renukswamy/Patent_sentiment_analysis', 'SetFit/tweet_sentiment_extraction', 'MonoHime/ru_sentiment_dataset', 'ebrigham/multi_sentiment', 'fhamborg/news_sentiment_newsmtsc', 'liam168/nlp_c4_sentiment', 'persiannlp/parsinlu_sentiment', 'rubrix/sentiment-banking', 'severo/autonlp-data-sentiment_detection-3c8bcd36', 'victor/autonlp-data-imdb-reviews-sentiment', 'winvoker/turkish-sentiment-analysis-dataset', 'chiarab/covid-tweet-sentiment', 'Davis/Swahili-tweet-sentiment', 'FanFan/sentiment-amazon-test', 'Marianina/sentiment-banking', 'ia-bentebib/tweet_eval_sentiment_fr', 'FanFan/sentiment-amazon-clean', 'chiarab/vaccine-sentiment-c

In [ ]:
from huggingface_hub import list_datasets
from datasets import load_dataset

# List available datasets
datasets_list = list(list_datasets())
print(f"Total datasets: {len(datasets_list)}")
datasets_list[:20]  # show first 20




HTTP Error 429 thrown while requesting GET https://huggingface.co/api/datasets?sort=trendingScore&cursor=eyIkb3IiOlt7InRyZW5kaW5nU2NvcmUiOjAsIl9pZCI6eyIkZ3QiOiI2ODc2NDhlNzY0MTNmNmVjODdhMjAwNzIifX0seyJ0cmVuZGluZ1Njb3JlIjp7IiRsdCI6MH19LHsidHJlbmRpbmdTY29yZSI6bnVsbH1dfQ%3D%3D
Retrying in 1s [Retry 1/20].
HTTP Error 429 thrown while requesting GET https://huggingface.co/api/datasets?sort=trendingScore&cursor=eyIkb3IiOlt7InRyZW5kaW5nU2NvcmUiOjAsIl9pZCI6eyIkZ3QiOiI2ODc2NDhlNzY0MTNmNmVjODdhMjAwNzIifX0seyJ0cmVuZGluZ1Njb3JlIjp7IiRsdCI6MH19LHsidHJlbmRpbmdTY29yZSI6bnVsbH1dfQ%3D%3D
Retrying in 2s [Retry 2/20].
HTTP Error 429 thrown while requesting GET https://huggingface.co/api/datasets?sort=trendingScore&cursor=eyIkb3IiOlt7InRyZW5kaW5nU2NvcmUiOjAsIl9pZCI6eyIkZ3QiOiI2ODc2NDhlNzY0MTNmNmVjODdhMjAwNzIifX0seyJ0cmVuZGluZ1Njb3JlIjp7IiRsdCI6MH19LHsidHJlbmRpbmdTY29yZSI6bnVsbH1dfQ%3D%3D
Retrying in 4s [Retry 3/20].
HTTP Error 429 thrown while requesting GET https://huggingface.co/api/datasets?sort=trendin

Total datasets: 498242


[DatasetInfo(id='HuggingFaceFW/finepdfs', author='HuggingFaceFW', sha='909850e0d03f191dced336f4e07d85f0fd1b9528', created_at=datetime.datetime(2025, 9, 5, 20, 8, 33, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2025, 9, 8, 13, 36, 38, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=51455, downloads_all_time=None, likes=418, paperswithcode_id=None, tags=['task_categories:text-generation', 'language:aai', 'language:aak', 'language:aau', 'language:aaz', 'language:aba', 'language:abi', 'language:abk', 'language:abn', 'language:abq', 'language:abs', 'language:abt', 'language:abx', 'language:aby', 'language:abz', 'language:aca', 'language:acd', 'language:ace', 'language:acf', 'language:ach', 'language:acm', 'language:acn', 'language:acr', 'language:acu', 'language:ada', 'language:ade', 'language:adh', 'language:adi', 'language:adj', 'language:adl', 'language:ady', 'language:adz', 'language:aeb', 'language:aer', 'language:aeu', 'language: